https://stashinvest.atlassian.net/browse/DSA-712

```sql
select
  stash_user_id,
  stash_user_uuid,
  last_collected_at,
  last_failed_at,
  money_link_last_initiated_at,
  is_passive_churn,
  coalesce(last_collected_at<last_failed_at, false)
    and coalesce(money_link_last_initiated_at<last_failed_at,true)
    as is_churn,
  not sub.has_active_subscription as is_active_churn
from
  mart.is_passive_churn full outer join
  mart.subscriptions as sub using (stash_user_id)
where (is_churn or is_active_churn) and is_churn is null
limit 500
```

1. integrated_users.platform_start_of_days
   1. bank_balance
   1. file_date
   1. invest_balance
   1. platform_balance
   1. process_date
   1. user_id
1. mart.subscriptions https://github.com/StashInvest/data-mart/blob/edge/stash/models/mart/subscriptions.sql
   1. stash_user_id	16112305
   1. has_active_subscription	true
   1. chargeable_at	2021-04-13T08:11:09.757+00:00
   1. subscribed_at	2021-04-13
   1. advisory_esig_at	∅
   1. is_onboarding_2	false
   1. last_selected_tier_name	premium
   1. ever_chargeable	true
   1. is_chargeable	true
   1. first_revenue_at	2021-04-14
   1. first_revenue_tier_name	Premium
   1. first_revenue_billing_period	monthly
   1. last_revenue_at	2021-11-12
   1. last_revenue_tier_name	Premium
   1. last_revenue_billing_period	monthly
   1. ever_fee_waiver	false
   1. attributed_acquisition_cost	99.44
   1. total_acquisition_cost	110.3316434169516
   1. contribution	6.496335
   1. subscriber_probability	0.8137
   1. leading_subscriber_probability	0.8137
   1. ever_paying_subscriber	1

# mart.is_passive_churn

https://github.com/StashInvest/data-mart/blob/edge/stash/models/staging/segment/is_passive_churn.sql

```sql
with

eligible_population as (
  -- Passive Churn Audience
  -- Only update this property for users who fully failed fee collection
  -- And have a current active subscription

  select    fees.stash_user_id,
            fees.stash_user_uuid,
            fees.last_collected_at,
            fees.last_failed_at

  from      ( -- subscription_fees table is unique per fee_id
            select    subscription_fees.stash_user_id,
                      subscription_fees.stash_user_uuid,
                      max(CASE WHEN status = 'succeeded' THEN fee_date END) as last_collected_at,
                      max(CASE WHEN status = 'failed' THEN fee_date END) as last_failed_at
            from      {{ref('subscription_fees')}} as subscription_fees
            where     subscription_fees.status IN ('failed', 'succeeded')
            group by  1,2
            ) as fees

  -- only update this value for people who have ever been charged before
  where     (last_collected_at IS NOT NULL OR last_failed_at IS NOT NULL)
),

phoenix_promo as (
  -- we want to toggle passive_churned based on whether they redeemed
  -- a phoenix promotion within the last 90 days
  select    stash_user_id,
            max(redeemed_at) as last_redeemed_phoenix_at
  from      {{ref('promotion')}}
  where     promotion_category in ('phoenix_reactivation','reactivation')
    and     redeemed_at is not null
  group by  1
),

last_initiated_link as (
  -- last initiated link's status in pending/completed/active only
  -- bank links get cancelled in 10 calendar days
  select    users.stash_user_id,
            MAX(external_bank_links.created_at) as last_bank_link_initiated_at,
            MAX(external_plastic_links.activated_at) as last_plastic_link_initiated_at
  from      {{ref('users')}}

  left join {{ref('external_bank_links')}}
    on      users.stash_user_id = external_bank_links.stash_user_id
    and     external_bank_links.aasm_state IN ('pending', 'approved')

  left join {{ref('external_plastic_links')}}
    on      users.stash_user_id = external_plastic_links.stash_user_id
    and     external_plastic_links.is_active
  group by 1
)

select    eligible_population.stash_user_id       as stash_user_id,
          eligible_population.stash_user_uuid     as stash_user_uuid,
          eligible_population.last_collected_at   as last_collected_at,
          eligible_population.last_failed_at      as last_failed_at,
          phoenix_promo.last_redeemed_phoenix_at  as last_redeemed_phoenix_at,
          last_bank_link_initiated_at             as last_bank_link_initiated_at,
          last_plastic_link_initiated_at          as last_plastic_link_initiated_at,
          greatest(
            link.last_bank_link_initiated_at,
            link.last_plastic_link_initiated_at
          )                                       as money_link_last_initiated_at,
          (
            coalesce( -- last collection was BEFORE the last failure
                last_collected_at < last_failed_at 
                OR  -- or last_collection_at is null, meaning they never paid
                last_collected_at IS NULL, false)

              and -- no payment link initiated after the failure
              coalesce(
                last_bank_link_initiated_at < last_failed_at
                AND
                last_plastic_link_initiated_at < last_failed_at, 
                true -- if null link initiations, then assume didn't link
              )

              and -- did not redeem a phoenix promo in the last 90 days
              coalesce(
                datediff( current_date, phoenix_promo.last_redeemed_phoenix_at ) > 90, 
                true
              )

              and -- remove active churn audience (only keep users with an active subscription)
              coalesce(sub.has_active_subscription, false)
          ) as is_passive_churn
from      eligible_population
left join {{ref('subscriptions')}} as sub 
  on      eligible_population.stash_user_id = sub.stash_user_id
left join phoenix_promo
  on      eligible_population.stash_user_id = phoenix_promo.stash_user_id
left join last_initiated_link as link 
  on      link.stash_user_id = eligible_population.stash_user_id
```